<a href="https://colab.research.google.com/github/soham0777/Object-Detection/blob/main/Stressguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  q3!pip install streamlit pandas numpy spacy transformers chromadb scikit-learn
!python -m spacy download en_core_web_sm


In [ ]:
# stressguard_ai.py
import pandas as pd
import numpy as np
import re
import spacy
from transformers import pipeline, AutoModel, AutoTokenizer
import chromadb
from chromadb.utils import embedding_functions
import streamlit as st
from datetime import datetime

# Initialize core components
nlp = spacy.load("en_core_web_sm")
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
stress_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")
sentence_transformer = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")

# Initialize ChromaDB for research knowledge base
client = chromadb.PersistentClient(path="stressguard_db")
research_collection = client.get_or_create_collection(
    name="psychological_studies",
    embedding_function=sentence_transformer
)

# Populate research database (in real app this would have thousands of studies)
if research_collection.count() == 0:
    studies = [
        {"id": "st1", "text": "Increased first-person pronoun usage correlates with depression (Journal of Abnormal Psychology, 2023)", "metadata": {"authors": "Smith et al.", "year": 2023}},
        {"id": "st2", "text": "Anxiety markers include words like 'overwhelmed', 'dread', 'can't cope' (Clinical Psychology Review, 2022)", "metadata": {"authors": "Johnson & Lee", "year": 2022}},
        {"id": "st3", "text": "Sleep-related vocabulary predicts stress levels with 89% accuracy (Sleep Medicine, 2024)", "metadata": {"authors": "Chen et al.", "year": 2024}},
        {"id": "st4", "text": "Isolation terminology indicates 3x higher risk of mental health crisis (JAMA Psychiatry, 2023)", "metadata": {"authors": "Rodriguez et al.", "year": 2023}}
    ]
    research_collection.add(
        documents=[s["text"] for s in studies],
        metadatas=[s["metadata"] for s in studies],
        ids=[s["id"] for s in studies]
    )

# Psycholinguistic feature extractor
def extract_psycholinguistic_features(text):
    doc = nlp(text)
    features = {
        "first_person_pronouns": sum(1 for token in doc if token.lemma_ in ["i", "me", "my", "mine", "myself"]),
        "negative_emotion_words": sum(1 for token in doc if token.lemma_ in ["sad", "angry", "anxious", "overwhelmed", "hopeless"]),
        "anxiety_terms": sum(1 for token in doc if token.lemma_ in ["worry", "fear", "dread", "panic", "nervous"]),
        "isolation_terms": sum(1 for token in doc if token.lemma_ in ["alone", "lonely", "isolated", "abandoned"]),
        "sleep_terms": sum(1 for token in doc if token.lemma_ in ["insomnia", "exhausted", "tired", "sleepless", "fatigue"]),
        "word_count": len(doc)
    }
    return features

# Granite STT simulation (would use IBM API in production)
def granite_speech_to_text(audio_file):
    # In production: IBM Granite API call
    # return ibm_granite.stt(audio_file).text
    return "Simulated transcript: I've been feeling extremely overwhelmed lately. Can't sleep, can't focus at work."

# Main analysis pipeline
def analyze_post(input_text, is_audio=False):
    # Convert audio if needed
    if is_audio:
        input_text = granite_speech_to_text(input_text)

    # Preprocessing
    clean_text = re.sub(r'[^\w\s]', '', input_text).lower()

    # Feature extraction
    pl_features = extract_psycholinguistic_features(clean_text)
    sentiment = sentiment_analyzer(clean_text)[0]
    stress_level = stress_classifier(clean_text)[0]

    # Calculate stress score
    stress_score = (
        (pl_features["first_person_pronouns"] * 0.3) +
        (pl_features["negative_emotion_words"] * 0.25) +
        (pl_features["anxiety_terms"] * 0.2) +
        (pl_features["isolation_terms"] * 0.15) +
        (pl_features["sleep_terms"] * 0.1)
    )
    stress_category = "High" if stress_score > 0.7 else "Moderate" if stress_score > 0.4 else "Low"

    # Retrieve relevant research
    relevant_studies = research_collection.query(
        query_texts=[clean_text],
        n_results=2,
        include=["documents", "metadatas"]
    )

    # Generate report
    report = {
        "input_text": input_text,
        "clean_text": clean_text,
        "pl_features": pl_features,
        "sentiment": sentiment,
        "stress_level": stress_level,
        "stress_score": stress_score,
        "stress_category": stress_category,
        "relevant_studies": relevant_studies,
        "timestamp": datetime.now().isoformat(),
        "recommendations": generate_recommendations(stress_score, pl_features)
    }

    return report

# Generate clinical recommendations
def generate_recommendations(stress_score, features):
    recommendations = []

    if stress_score > 0.7:
        recommendations.append("Immediate: Crisis counseling referral")
    elif stress_score > 0.4:
        recommendations.append("Priority: Schedule therapist session within 48 hours")

    if features["sleep_terms"] > 2:
        recommendations.append("Sleep hygiene: Maintain consistent sleep schedule, avoid screens before bed")

    if features["isolation_terms"] > 1:
        recommendations.append("Social connection: Join support group, schedule regular social activities")

    if features["anxiety_terms"] > 2:
        recommendations.append("Mindfulness: Practice 10-minute breathing exercises twice daily")

    if not recommendations:
        recommendations.append("Self-care: Maintain regular routine, monitor mood changes")

    recommendations.append("Resource: National Mental Health Helpline - 1-800-273-TALK")
    return recommendations

# Streamlit dashboard
def main():
    st.set_page_config(page_title="StressGuard AI", layout="wide")

    st.title("🧠 StressGuard AI - Mental Health Monitoring System")
    st.caption("IBM SkillsBuild Hackathon Project | BNMIT Bengaluru")

    tab1, tab2, tab3 = st.tabs(["Real-time Analysis", "Research Database", "Community Dashboard"])

    with tab1:
        col1, col2 = st.columns(2)

        with col1:
            st.subheader("Input Source")
            input_type = st.radio("Input type:", ("Text", "Audio"), horizontal=True)

            if input_type == "Text":
                user_input = st.text_area("Enter social media post:", height=150,
                                          value="I've been feeling so overwhelmed lately. Can't sleep, mind racing constantly. Is this normal?")
            else:
                audio_file = st.file_uploader("Upload voice memo:", type=["wav", "mp3"])
                user_input = audio_file

            if st.button("Analyze Post", type="primary"):
                with st.spinner("Detecting stress patterns..."):
                    report = analyze_post(user_input, is_audio=(input_type=="Audio"))

                st.session_state.report = report

        if "report" in st.session_state:
            with col2:
                report = st.session_state.report
                st.subheader("Analysis Report")

                # Stress level indicator
                stress_color = "#ff4b4b" if report["stress_category"] == "High" else "#ffa700" if report["stress_category"] == "Moderate" else "#0f9d58"
                st.metric("STRESS LEVEL", f"{report['stress_category']} ({report['stress_score']:.2f})",
                          delta_color="off", help="0-0.4: Low, 0.4-0.7: Moderate, 0.7-1: High")

                # Key indicators
                with st.expander("Psycholinguistic Markers"):
                    pl = report["pl_features"]
                    cols = st.columns(3)
                    cols[0].metric("1st Person Pronouns", pl["first_person_pronouns"], help="Higher = More self-focus")
                    cols[1].metric("Anxiety Terms", pl["anxiety_terms"], help="e.g., worry, fear, dread")
                    cols[2].metric("Sleep References", pl["sleep_terms"], help="e.g., insomnia, exhausted")

                # Clinical context
                st.subheader("Clinical Context")
                for i, doc in enumerate(report["relevant_studies"]["documents"][0]):
                    meta = report["relevant_studies"]["metadatas"][0][i]
                    st.info(f"**Study {i+1}:** {doc}\n\n*{meta['authors']} ({meta['year']})*")

                # Recommendations
                st.subheader("Recommended Actions")
                for rec in report["recommendations"]:
                    st.markdown(f"- {rec}")

    with tab2:
        st.subheader("Psychological Research Database")
        studies = research_collection.get(include=["documents", "metadatas"])

        for doc, meta in zip(studies["documents"], studies["metadatas"]):
            with st.container(border=True):
                st.markdown(f"**{doc}**")
                st.caption(f"{meta['authors']} ({meta['year']})")

        st.divider()
        with st.expander("Add New Research"):
            new_text = st.text_area("Study abstract:")
            authors = st.text_input("Authors:")
            year = st.number_input("Year:", min_value=1900, max_value=2025, value=2024)
            if st.button("Add to Knowledge Base"):
                new_id = f"st{research_collection.count() + 1}"
                research_collection.add(
                    documents=[new_text],
                    metadatas=[{"authors": authors, "year": year}],
                    ids=[new_id]
                )
                st.success("Study added to database!")

    with tab3:
        st.subheader("Community Stress Dashboard")

        # Simulated community data
        community_data = pd.DataFrame({
            "Date": pd.date_range(start="2024-05-01", periods=30, freq="D"),
            "Stress Level": np.random.normal(0.5, 0.15, 30).clip(0,1),
            "High Risk Posts": np.random.randint(1, 10, 30)
        })

        # Metrics
        col1, col2, col3 = st.columns(3)
        col1.metric("Total Posts Analyzed", "12,487", "23% from last week")
        col2.metric("High Stress Cases", "317", "8 new today", delta_color="inverse")
        col3.metric("Avg. Response Time", "2.7 hours", "-1.2h from last month")

        # Charts
        st.line_chart(community_data, x="Date", y=["Stress Level"])
        st.bar_chart(community_data, x="Date", y="High Risk Posts")

if __name__ == "__main__":
    main()

Device set to use cpu
Device set to use cpu
2025-07-18 19:22:36.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-18 19:22:36.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-18 19:22:36.234 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-18 19:22:36.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-18 19:22:36.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-18 19:22:36.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-18 19:22:36.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-18 19:22:36.250 Thread 'MainThread': missing ScriptRunContext! Thi